In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import math
import os
import sys
import warnings
import argparse
import numpy as np
import yaml
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset,Sampler
from torchvision import datasets,transforms
import torchvision.transforms as transforms
from torch.nn import functional as F

sys.path.append("/home/shiyi/gpu/gpu/")
sys.path.append("/home/shiyi/colposcopy/colposcopy/util")
img_dir = "/data/colposcopy/colposcopy"
img_dir_target = "/data/colposcopy/colposcopy_1"

from gpu_allocation import set_gpu


error = glob(os.path.join(img_dir,"train*","*","2845.jpg"))+ \
    glob(os.path.join(img_dir,"train*","*","5893.jpg"))+glob(os.path.join(img_dir,"train*","*","5892.jpg"))
print(error)

from gpu_allocation import set_gpu
corrupt_set  = set()
types = [1,2,3]

type_1_train = [i for i in glob(os.path.join(img_dir,"train*","Type_1","*.jpg")) if i not in error]
type_2_train = [i for i in glob(os.path.join(img_dir,"train*","Type_2","*.jpg")) if i not in error]
type_3_train = [i for i in glob(os.path.join(img_dir,"train*","Type_3","*.jpg")) if i not in error]

img_test   = glob(os.path.join(img_dir,"test","*.jpg"))
img_test_wo_answer = glob(os.path.join(img_dir,"test_stg2","*.jpg"))

img_paths = type_1_train+type_2_train+type_3_train+img_test+img_test_wo_answer


[u'/data/colposcopy/colposcopy/train_stg2/Type_2/2845.jpg', u'/data/colposcopy/colposcopy/train_stg2/Type_1/5893.jpg', u'/data/colposcopy/colposcopy/train_stg2/Type_2/5892.jpg']


In [2]:
class ColpoProcessDataset(Dataset):
    def __init__(self):
        pass
    def __getitem__(self, index):
        path = img_paths[index]
        x = cv2.imread(path)
        if x is None:
            return  None,None
        return x
    def __len__(self):
        return len(img_paths)
    
def cvt(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def preprocess(img): 
    r_c = img[:,:,2]
    g_c = img[:,:,1]
    b_c = img[:,:,0]
    _,r_thres = cv2.threshold(r_c, 210, 255, cv2.THRESH_BINARY)
    _,g_thres = cv2.threshold(g_c, 208, 255, cv2.THRESH_BINARY)
    _,b_thres = cv2.threshold(b_c, 206, 255, cv2.THRESH_BINARY)
    mixed = r_thres & b_thres & g_thres
    kernel_dilate = np.ones((18, 18), np.uint8)
    mask = cv2.dilate(mixed,kernel_dilate)
    dst = cv2.inpaint(img,mask,14,cv2.INPAINT_TELEA)
    return cvt(dst)

def collate_fn1(batch):
    x = [item[0] for item in batch if item[0] is not None]
    y = [item[1] for item in batch if item[0] is not None]
    return x,y

In [3]:
print(img_paths[0].split('/')[4:])

[u'train', u'Type_1', u'732.jpg']


In [6]:
batch_size_train = 16
colpoProcessDataset = ColpoProcessDataset()
process_loader = DataLoader(dataset=colpoProcessDataset,batch_size=batch_size_train,shuffle=True,num_workers = 2)
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
#num_gpu = 1
#set_gpu(num_gpu)
    
for img_path in tqdm(img_paths):
    img_path_target = img_dir_target + '/'+'/'.join(img_path.split('/')[4:])
    print(img_path_target)
    img = cv2.imread(img_path)
    x = preprocess(img)
    cv2.imwrite(img_path_target,x,[int(cv2.IMWRITE_JPEG_QUALITY),100])


  0%|          | 0/12230 [00:00<?, ?it/s]

/data/colposcopy/colposcopy_1/train/Type_1/732.jpg



  0%|          | 1/12230 [00:05<16:59:43,  5.00s/it]

/data/colposcopy/colposcopy_1/train/Type_1/764.jpg



  0%|          | 2/12230 [00:49<57:22:49, 16.89s/it]

/data/colposcopy/colposcopy_1/train/Type_1/1071.jpg



  0%|          | 3/12230 [00:59<49:46:10, 14.65s/it]

/data/colposcopy/colposcopy_1/train/Type_1/298.jpg


KeyboardInterrupt: 